In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy
from numba import njit
from numba import jit
from numba import vectorize
from timeit import default_timer as timer
from datetime import timedelta

In [2]:
movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
%%time
tfidf = CountVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['title'].apply(str) + " " + movies['overview'].apply(str))
tfidf_matrix

Wall time: 15.4 s


<45466x2407186 sparse matrix of type '<class 'numpy.int64'>'
	with 3943994 stored elements in Compressed Sparse Row format>

In [10]:
userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))
movieid_list = np.array(movieid_list)

# get all movie vector rated from user
def get_user_rated_movies(tmdbs):
    movie_vector_list = []
    for i,id in enumerate(tmdbs):
        index = np.where(movieid_list == id)
        movie_vector = tfidf_matrix[index]
        movie_vector_list.append(scipy.sparse.csr_matrix.toarray(movie_vector)) 
        
    movie_vectors = np.array(movie_vector_list)
    #movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


# get rated movies from user, convert mid to tmdbid
def get_user_profile(uid):
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    user_movieid_list = user_profile['movieId'].to_list()
    user_movieid_tmdbId = np.zeros_like(user_movieid_list)
    for i, mid in enumerate(user_movieid_list):
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == uid]
        try:
            tmdbId = int(tmdbId)
            user_movieid_tmdbId[i] = tmdbId
        except Exception:
            print ("ValueError tmdbId: " + str(tmdbId))

    return user_movieid_tmdbId


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector    
        

# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}
def create_user_vectors():
    for uid in userid_list:
        user_tmdbid_array = get_user_profile(uid)
        user_rated_movies_vectors = get_user_rated_movies(user_tmdbid_array)

        print(type(user_rated_movies_vectors))
        print(user_rated_movies_vectors.size)
        n = len(user_tmdbid_array)
        if n < 200:
            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()
            user_vector = create_user_vector_jit(user_rated_movies_vectors)
            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = scipy.sparse.csr_matrix(user_norm)
            print(str(uid) + " finished\n")
        else:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()
            user_rated_movies_vector_csr = scipy.sparse.csr_matrix(user_rated_movies_vectors)
            user_vector = user_rated_movies_vector_csr[0]
            for x in user_rated_movies_vector_csr[1:]:
                user_vector = user_vector + x
            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")
        

%time create_user_vectors()

<class 'numpy.ndarray'>
48143720
Jit Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:00.783694
1 finished

<class 'numpy.ndarray'>
182946136
Jit Time Compute User Vector: n = 76
Execution time HH:MM:SS:  0:00:00.549685
2 finished

<class 'numpy.ndarray'>
122766486
Jit Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.352692
3 finished

<class 'numpy.ndarray'>
491065944
CSR Time Compute User Vector: n = 204


TypeError: expected dimension <= 2 array or matrix

In [30]:
# VERSION 2, n = 500

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector 


# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]
        
        if n > 500:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")

        else:
            user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
            for i in range(n):
                user_vectors_array[i] = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list[i])

            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = create_user_vector_jit(user_vectors_array)

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")


%time create_user_vectors()

= 126
Execution time HH:MM:SS:  0:00:00.834060
496 finished

Jit Time Compute User Vector: n = 105
Execution time HH:MM:SS:  0:00:00.729646
497 finished

Jit Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:00.118386
498 finished

Jit Time Compute User Vector: n = 33
Execution time HH:MM:SS:  0:00:00.211236
499 finished

ValueError tmdbId: 5204    164721.0
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 248
Execution time HH:MM:SS:  0:00:01.774064
500 finished

Jit Time Compute User Vector: n = 203
Execution time HH:MM:SS:  0:00:01.474396
501 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 112
Execution time HH:MM:SS:  0:00:00.730164
502 finished

Jit Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.268738
503 finished

Jit Time Compute User Vector: n = 24
Execution time HH:MM:SS:  0:00:00.151666
504 finished

Jit Time Compute User Vector: n = 395
Execution time HH:MM:SS:  0:00:03.1030

n = 200 -> 26.5 min <br>
n = 500 -> 29.16 min <br>
n = 700 -< 29.25 min <br>
n = 700 different timer -< 32.14 min <br>
n = 700, better csr to array -< 22.07 min <br>
n = 1000, better csr to array -< 19.25 min <br>
n = 1000, better csr to array, ssd enabled ram -< 18.50 min <br>
n = 1000, better csr to array, ssd enabled ram, sum(axis=0)  -< 15.29 min <br>

In [31]:
# VERSION 2 , n= 700

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector 


# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]
        
        if n > 700:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")

        else:
            user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
            for i in range(n):
                user_vectors_array[i] = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list[i])

            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = create_user_vector_jit(user_vectors_array)

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")


%time create_user_vectors()

= 126
Execution time HH:MM:SS:  0:00:00.803896
496 finished

Jit Time Compute User Vector: n = 105
Execution time HH:MM:SS:  0:00:00.662545
497 finished

Jit Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:00.120822
498 finished

Jit Time Compute User Vector: n = 33
Execution time HH:MM:SS:  0:00:00.193430
499 finished

ValueError tmdbId: 5204    164721.0
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 248
Execution time HH:MM:SS:  0:00:01.801789
500 finished

Jit Time Compute User Vector: n = 203
Execution time HH:MM:SS:  0:00:01.319156
501 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 112
Execution time HH:MM:SS:  0:00:00.694147
502 finished

Jit Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.265065
503 finished

Jit Time Compute User Vector: n = 24
Execution time HH:MM:SS:  0:00:00.141961
504 finished

Jit Time Compute User Vector: n = 395
Execution time HH:MM:SS:  0:00:02.9856

In [35]:
# VERSION 2 , n= 700, different timer

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector 


# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]
        
        if n > 700:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")

        else:
            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()

            #user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
            user_vectors_array = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list)
            user_vectors_array.reshape(user_rated_movies_vector_list.shape)

            user_vector = create_user_vector_jit(user_vectors_array)

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")


%time create_user_vectors()

Jit Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:03.450170
1 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 75
Execution time HH:MM:SS:  0:00:00.547125
2 finished

Jit Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.420160
3 finished

Jit Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.608784
4 finished

Jit Time Compute User Vector: n = 100
Execution time HH:MM:SS:  0:00:01.108949
5 finished

Jit Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.469198
6 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 87
Execution time HH:MM:SS:  0:00:00.653616
7 finished

Jit Time Compute User Vector: n = 116
Execution time HH:MM:SS:  0:00:00.970118
8 finished

Jit Time Compute User Vector: n = 45
Execution time HH:MM:SS:  0:00:00.467000
9 finished

Jit Time Compute User Vector: n = 46
Execution time HH:MM:SS:  0:00:00.

In [36]:
# VERSION 2 , n= 1000, different timer

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector 


# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]
        
        if n > 1000:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")

        else:
            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()

            #user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
            user_vectors_array = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list)
            user_vectors_array.reshape(user_rated_movies_vector_list.shape)

            user_vector = create_user_vector_jit(user_vectors_array)

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")


%time create_user_vectors()

Jit Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:02.653458
1 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 75
Execution time HH:MM:SS:  0:00:00.568206
2 finished

Jit Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.455684
3 finished

Jit Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.655729
4 finished

Jit Time Compute User Vector: n = 100
Execution time HH:MM:SS:  0:00:01.121359
5 finished

Jit Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.496059
6 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 87
Execution time HH:MM:SS:  0:00:00.802804
7 finished

Jit Time Compute User Vector: n = 116
Execution time HH:MM:SS:  0:00:01.247792
8 finished

Jit Time Compute User Vector: n = 45
Execution time HH:MM:SS:  0:00:00.605134
9 finished

Jit Time Compute User Vector: n = 46
Execution time HH:MM:SS:  0:00:00.

In [5]:
# VERSION 2 , n= 1000, different timer, SSD enabled

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector 


# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]
        
        if n > 1000:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")

        else:
            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()

            #user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
            user_vectors_array = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list)
            user_vectors_array.reshape(user_rated_movies_vector_list.shape)

            user_vector = create_user_vector_jit(user_vectors_array)

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")


%time create_user_vectors()

Jit Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:00.674572
1 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 75
Execution time HH:MM:SS:  0:00:00.558631
2 finished

Jit Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.518659
3 finished

Jit Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.582623
4 finished

Jit Time Compute User Vector: n = 100
Execution time HH:MM:SS:  0:00:01.113895
5 finished

Jit Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.429718
6 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 87
Execution time HH:MM:SS:  0:00:00.886228
7 finished

Jit Time Compute User Vector: n = 116
Execution time HH:MM:SS:  0:00:01.034496
8 finished

Jit Time Compute User Vector: n = 45
Execution time HH:MM:SS:  0:00:00.488553
9 finished

Jit Time Compute User Vector: n = 46
Execution time HH:MM:SS:  0:00:00.

In [6]:
# VERSION 2 , n= 1000 all jit, different timer, SSD, sum(axis=0)

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    #user_vector = user_rated_movies_vector_array[0]

    #for x in user_rated_movies_vector_array[1:]:
    #    user_vector = user_vector + x
    
    user_vector = user_rated_movies_vector_array.sum(axis=0)
    return user_vector

# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]
        
        if n > 1000:
            print("CSR Time Compute User Vector: n = " + str(n))
            start = timer()

            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")

        else:
            print("Jit Time Compute User Vector: n = " + str(n))
            start = timer()

            #user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
            user_vectors_array = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list)
            user_vectors_array.reshape(user_rated_movies_vector_list.shape)

            user_vector = create_user_vector_jit(user_vectors_array)

            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")


%time create_user_vectors()

Jit Time Compute User Vector: n = 20
C:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\numba\core\typed_passes.py:326: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-6-f261459cf171>", line 29:
@njit(fastmath=True, parallel=True)
def create_user_vector_jit(user_rated_movies_vector_array):
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,
Execution time HH:MM:SS:  0:00:01.123992
1 finished

ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Jit Time Compute User Vector: n = 75
Execution time HH:MM:SS:  0:00:00.491695
2 finished

Jit Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.345023
3 finished

Jit Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.240136
4 finished


In [18]:
tmdbId = 280

print("Time Compute User Vector np.where: " )
start = timer()
index = np.where(movieid_list == tmdbId)
end = timer()
print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))


print("Time Compute User Vector list.index: " )
start = timer()
index = movieid_list.index(tmdbId)
end = timer()
print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

Time Compute User Vector np.where: 
Execution time HH:MM:SS:  0:00:00.000127
Time Compute User Vector list.index: 
Execution time HH:MM:SS:  0:00:00.000122


In [4]:
from concurrent.futures import ThreadPoolExecutor
import threading

In [5]:
# Threading
# VERSION 2 , n= 1000 all jit, different timer, SSD, sum(axis=0)

userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))


# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            x
            #print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies


    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


@njit(fastmath=True)
def create_user_vector_jit(user_rated_movies_vector_array):
    
    user_vector = user_rated_movies_vector_array.sum(axis=0)

    return user_vector

# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors(uid):

    global all_user_profiles
    
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        
    user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
    n = user_rated_movies_vector_list.shape[0]
        
    if n > 1000:
        #print("CSR Time Compute User Vector: n = " + str(n))
        #start = timer()

        user_vector = user_rated_movies_vector_list[0]
        for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

        #end = timer()
        #print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

        user_norm = sklearn.preprocessing.normalize(user_vector)
        all_user_profiles[uid] = user_norm
        print(str(uid) + " finished")

    else:
        #print("Jit Time Compute User Vector: n = " + str(n))
        #start = timer()

        #user_vectors_array = np.zeros(user_rated_movies_vector_list.shape)
        user_vectors_array = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list)
        user_vectors_array.reshape(user_rated_movies_vector_list.shape)

        user_vector = create_user_vector_jit(user_vectors_array)

        #end = timer()
        #print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            
        user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
        all_user_profiles[uid] = user_norm
        #print(str(uid) + " finished")



start = timer()

with ThreadPoolExecutor(max_workers=8) as executer:
    executer.map(create_user_vectors, userid_list)

end = timer()
print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))



452 finished
Execution time HH:MM:SS:  0:09:00.748532


In [39]:
Time Threading = 9.20 min
Time Threading 4 Worker = 10.10 min
Time Threading 8 Worker = 9.24 min


